# Applying PB-M3C2 in long term monitoring

In applications where data from the same observation site is acquired over a long period of time, it is desirable to carry out the training of the PB-M3C2 algorithm once and then apply the trained model to newly acquired epochs. This notebook explains how this process is implemented in `py4dgeo`. First, we are carrying out the training procedure like we did in the explanation of the [base workflow](PB_M3C2.ipynb):

In [ ]:
import py4dgeo

In [ ]:
py4dgeo.set_interactive_backend("vtk")

In [ ]:
py4dgeo.ensure_test_data_availability()

In [ ]:
epoch0, epoch1 = py4dgeo.read_from_xyz(
    "plane_horizontal_t1.xyz", "plane_horizontal_t2.xyz"
)

In [ ]:
alg = py4dgeo.PB_M3C2()

In [ ]:
xyz_epoch0, xyz_epoch1, segment_id = alg.export_segmented_point_cloud_and_segments(
    epoch0=epoch0,
    epoch1=epoch1,
)

In [ ]:
alg.training(
    extracted_segments_file_name="extracted_segments.seg",
    extended_y_file_name="testdata-labelling.csv",
)

Having the pre-trained algorithm object `alg`, we would like to save it for reuse in later analysis sessions. We do use Python's `pickle` module for that:

In [ ]:
import pickle

In [ ]:
with open("alg.pickle", "wb") as outfile:
    pickle.dump(alg, outfile)

Then, in a subsequent session, we can reload the algorithm using `pickle`:

In [ ]:
with open("alg.pickle", "rb") as infile:
    alg = pickle.load(infile)

We can then feed new epochs (here, we just use `epoch0` again) into the algorithm. It will apply segmentation on the new epoch and then run the prediction for the new epoch

In [ ]:
(
    _0,
    _1,
    extracted_segments_epoch0,
) = alg.export_segmented_point_cloud_and_segments(
    # must be a new epoch
    epoch0=epoch0,
    # epoch1=None,
    x_y_z_id_epoch0_file_name=None,
    x_y_z_id_epoch1_file_name=None,
    extracted_segments_file_name=None,
)

We can then calculate distances for the new epoch. Note, that in order to disable those parts of the analysis pipeline that are already computed for the reference epoch, we pass the constant dictionary `**py4dgeo.config_epoch0_as_segments`. If you have customized the analysis pipeline, you should adapt the configuration settings accordingly and disable all those steps that are not required for the reference epoch:

In [ ]:
distances, uncertainties = alg.compute_distances(
    epoch0=extracted_segments_epoch0, epoch1=epoch1, **py4dgeo.config_epoch0_as_segments
)